# Convertisseur et Traducteur de fichiers ePub

Ce notebook permet de :
1. Convertir des fichiers ePub en DOCX
2. Traduire les fichiers DOCX en français
3. Reconvertir les fichiers traduits en ePub

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install ebooklib beautifulsoup4 googletrans==3.1.0a0 html2text psutil deep-translator>=1.11.4
#!pip install -r /content/requirements.txt
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=ea900f25b2978113602c6b807dbd9056fc21206aad2efc1c81fa833bcde0fac9
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succe

In [ ]:
import os
import glob
import shutil
import zipfile
from bs4 import BeautifulSoup
from googletrans import Translator
import time
import json
import hashlib

def ensure_directories():
    """Crée les dossiers nécessaires"""
    directories = [
        '/content/drive/MyDrive/fanfictions/vo',
        '/content/drive/MyDrive/fanfictions/temp/extraction',
        '/content/drive/MyDrive/fanfictions/temp/vf',
        '/content/drive/MyDrive/fanfictions/vf',
        '/content/drive/MyDrive/fanfictions/logs'
    ]
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

class TranslationCache:
    def __init__(self):
        self.cache_file = '/content/drive/MyDrive/fanfictions/logs/translation_cache.json'
        self.cache = self._load_cache()

    def _load_cache(self):
        """Charge le cache depuis le fichier"""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except Exception as e:
                print(f"Erreur lors du chargement du cache: {e}")
                return {}
        return {}

    def _save_cache(self):
        """Sauvegarde le cache dans le fichier"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, ensure_ascii=False, indent=2)
        except Exception as e:
            print(f"Erreur lors de la sauvegarde du cache: {e}")

    def get_translation(self, text):
        """Récupère une traduction du cache"""
        text_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
        return self.cache.get(text_hash)

    def add_translation(self, text, translation):
        """Ajoute une traduction au cache"""
        text_hash = hashlib.md5(text.encode('utf-8')).hexdigest()
        self.cache[text_hash] = translation
        self._save_cache()

def extract_epub(epub_path, extract_dir):
    """Extrait le contenu du fichier ePub"""
    with zipfile.ZipFile(epub_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Fichier extrait dans {extract_dir}")

def find_xhtml_files(directory):
    """Trouve tous les fichiers XHTML dans le dossier et ses sous-dossiers"""
    xhtml_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.xhtml', '.html', '.htm')):
                xhtml_files.append(os.path.join(root, file))
    return xhtml_files

def translate_xhtml_file(input_path, output_path, translation_cache):
    """Traduit un fichier XHTML en utilisant le cache"""
    print(f"Traduction de {os.path.basename(input_path)}")
    translator = Translator()

    # Lire le fichier source
    with open(input_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Parser le contenu HTML
    soup = BeautifulSoup(content, 'html.parser')

    # Sauvegarder les styles
    style_tags = soup.find_all('style')
    original_styles = ''.join(str(style) for style in style_tags)

    # Traduire tous les éléments de texte
    elements_translated = 0
    elements_cached = 0

    for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'div', 'span']):
        if element.string and element.string.strip():
            original_text = element.string.strip()

            # Vérifier dans le cache
            cached_translation = translation_cache.get_translation(original_text)
            if cached_translation:
                element.string = cached_translation
                elements_cached += 1
                continue

            try:
                # Traduire le texte
                translated = translator.translate(original_text, dest='fr')
                element.string = translated.text
                # Ajouter au cache
                translation_cache.add_translation(original_text, translated.text)
                elements_translated += 1
                # Petit délai pour éviter les limitations de l'API
                time.sleep(0.5)
            except Exception as e:
                print(f"Erreur de traduction: {e}")

    print(f"Éléments traduits: {elements_translated}, Éléments trouvés dans le cache: {elements_cached}")

    # Sauvegarder le fichier traduit
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(str(soup))

def create_epub(input_dir, output_path):
    """Crée un nouveau fichier ePub à partir des fichiers du dossier"""
    print(f"Création du fichier ePub: {output_path}")

    # Créer le fichier ZIP (ePub)
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        # Ajouter le mimetype sans compression
        mimetype_path = os.path.join(input_dir, 'mimetype')
        if os.path.exists(mimetype_path):
            zf.write(mimetype_path, 'mimetype', compress_type=zipfile.ZIP_STORED)

        # Ajouter tous les autres fichiers
        for root, _, files in os.walk(input_dir):
            for file in files:
                if file != 'mimetype':  # Skip mimetype as it's already added
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, input_dir)
                    zf.write(file_path, arcname)

def clean_temp_directories():
    """Nettoie les dossiers temporaires"""
    temp_dirs = ['/content/drive/MyDrive/fanfictions/temp/extraction', '/content/drive/MyDrive/fanfictions/temp/vf']
    for dir_path in temp_dirs:
        if os.path.exists(dir_path):
            shutil.rmtree(dir_path)
            os.makedirs(dir_path)
    print("Dossiers temporaires nettoyés")

def process_files():
    """Traite tous les fichiers ePub dans le dossier d'entrée"""
    ensure_directories()
    translation_cache = TranslationCache()
    epub_files = glob.glob('/content/drive/MyDrive/fanfictions/vo/*.epub')

    for epub_file in epub_files:
        try:
            base_name = os.path.basename(epub_file)
            name_without_ext = os.path.splitext(base_name)[0]

            # Vérifier si le fichier a déjà été traduit
            output_epub = f'/content/drive/MyDrive/fanfictions/vf/{name_without_ext}_fr.epub'
            if os.path.exists(output_epub):
                print(f"Le fichier {base_name} a déjà été traduit. Passage au suivant...")
                continue

            print(f'Traitement de {base_name}...')

            # Nettoyer le dossier d'extraction
            extraction_dir = '/content/drive/MyDrive/fanfictions/temp/extraction'
            if os.path.exists(extraction_dir):
                shutil.rmtree(extraction_dir)
            os.makedirs(extraction_dir)

            # 1. Extraire l'ePub
            extract_epub(epub_file, extraction_dir)

            # 2. Trouver et traduire les fichiers XHTML
            xhtml_files = find_xhtml_files(extraction_dir)
            for xhtml_file in xhtml_files:
                # Créer le chemin de sortie en préservant la structure des dossiers
                rel_path = os.path.relpath(xhtml_file, extraction_dir)
                output_path = os.path.join('/content/drive/MyDrive/fanfictions/temp/vf', rel_path)

                # Traduire le fichier
                translate_xhtml_file(xhtml_file, output_path, translation_cache)

                # Remplacer le fichier original par la version traduite
                shutil.copy2(output_path, xhtml_file)

            # 3. Créer le nouveau fichier ePub
            create_epub(extraction_dir, output_epub)

            print(f'Terminé! Fichier traduit sauvegardé: {output_epub}')

            # 4. Nettoyer les dossiers temporaires
            clean_temp_directories()

        except Exception as e:
            print(f"Erreur lors du traitement de {base_name}: {str(e)}")
            # Nettoyer même en cas d'erreur
            clean_temp_directories()
            continue

if __name__ == "__main__":
    process_files()

Le fichier Rogue_Knight_by_Illuviar-XhqrMJNx.epub a déjà été traduit. Passage au suivant...
Le fichier Harry_Potter_and_the_Second_Chance_by_PropheticScript-kz9l63yg.epub a déjà été traduit. Passage au suivant...
Le fichier Meant_to_be_by_vedakshu2006-bfxypana.epub a déjà été traduit. Passage au suivant...
Le fichier Enchanting_Melodies_by_athassprkr-azjeosdz.epub a déjà été traduit. Passage au suivant...
Le fichier Defrost_by_1Valor1-3o5a79mz.epub a déjà été traduit. Passage au suivant...
Le fichier HARRY_POTTER_THE_LOST_LEGACY_by_peverell_magic-ruvkkh9s.epub a déjà été traduit. Passage au suivant...
Le fichier The_Chosen_Six_by_Tribun-kBnFE6Zd.epub a déjà été traduit. Passage au suivant...
Le fichier Harry_Potter_and_Sirius_Legacy_by_C_A_Rotwang-p6xrbvef.epub a déjà été traduit. Passage au suivant...
Le fichier A_Promise_Given_by_Umthieral-isd4thls.epub a déjà été traduit. Passage au suivant...
Le fichier Cavorting_with_Death_by_Rokakku-fsidv6jf.epub a déjà été traduit. Passage au su